In [3]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
import os 
import random 
import cv2
from detectron2.utils.visualizer import Visualizer 
from detectron2.engine import DefaultTrainer 
from detectron2.config import get_cfg 

from detectron2.engine.defaults import DefaultPredictor

# 注册数据，获取元数据以及数据集。如果数据集格式是coco格式可以按下面方法注册数据集
# 不然的话需要DatasetCatalog.register(my_dataset_name, get_dicts)这样，其中get_dicts是一个包含数据的数组
register_coco_instances("car_street", {}, "../Data/carStreet/train.json", "../Data/carStreet/images")
register_coco_instances("car_street_test", {}, "../Data/carStreet/val.json", "../Data/carStreet/val")

car_metadata = MetadataCatalog.get("car_street")
dataset_dicts = DatasetCatalog.get("car_street")
dataset_dicts_test = DatasetCatalog.get("car_street_test")

AssertionError: Dataset 'car_street' is already registered!

In [4]:
# 随机显示几个数据，带有mask框
for d in random.sample(dataset_dicts, 3): 
    img = cv2.imread(d["file_name"]) 
    visualizer = Visualizer(img[:, :, ::-1], metadata=car_metadata, scale=0.5) 
    vis = visualizer.draw_dataset_dict(d) 
    cv2.imshow("test", vis.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [29]:

# 准备训练模型， 创建cfg模型的配置文件
cfg = get_cfg() 
cfg.merge_from_file("../study/detectron2-master/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml" ) 
cfg.DATASETS.TRAIN = ("car_street",) 
cfg.DATASETS.TEST = () # no metrics implemented for this dataset 
cfg.DATALOADER.NUM_WORKERS = 2 
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02 
cfg.SOLVER.MAX_ITER = ( 300 ) # 迭代次数
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = ( 128 ) # faster, and good enough for this toy dataset 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # 类别的个数

os.makedirs("./output/carStreet", exist_ok=True) 
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) 
trainer.train()

Config '../study/detectron2-master/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
'roi_heads.box_predictor.cls_score.weight' has shape (81, 1024) in the checkpoint but (2, 1024) in the model! Skipped.
'roi_heads.box_predictor.cls_score.bias' has shape (81,) in the checkpoint but (2,) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.weight' has shape (320, 1024) in the checkpoint but (4, 1024) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.bias' has shape (320,) in the checkpoint but (4,) in the model! Skipped.
'roi_heads.mask_head.predictor.weight' has shape (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in the model! Skipped.
'roi_heads.mask_head.predictor.bias' has shape (80,) in the checkpoint but (1,) in the model! Skipped.


OrderedDict()

In [31]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # 加载模型
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("car_street", )
predictor = DefaultPredictor(cfg)

In [34]:
from detectron2.utils.visualizer import ColorMode 

for d in random.sample(dataset_dicts, 5):   
    im = cv2.imread(d["file_name"]) 
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1], 
                   metadata=car_metadata,
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW # remove the colors of unsegmented pixels  
                  ) 
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))    
    cv2.imshow("test", v.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [30]:
%load_ext tensorboard
%tensorboard --logdir output

Reusing TensorBoard on port 6006 (pid 12538), started 6:26:16 ago. (Use '!kill 12538' to kill it.)

In [36]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # 加载模型
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("car_street_test", )
predictor = DefaultPredictor(cfg)
from detectron2.utils.visualizer import ColorMode 

for d in random.sample(dataset_dicts_test, 15):   
    im = cv2.imread(d["file_name"]) 
    outputs = predictor(im) 
    v = Visualizer(im[:, :, ::-1], 
                   metadata=car_metadata,
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW # remove the colors of unsegmented pixels  
                  ) 
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))    
    cv2.imshow("test", v.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows() 